In [ ]:
import torch
import matplotlib.pyplot as plt
from nerfstudio.utils.eval_utils import eval_setup
from pathlib import Path
import numpy as np
# config = Path("outputs/garfield_plushie/dig/2024-02-27_144117/config.yml")#pca 64
# config = Path("outputs/garfield_plushie/dig/2024-02-27_163355/config.yml")#pca 128
# config = Path("outputs/garfield_plushie/dig/2024-02-27_164651/config.yml")#v2 128 dim
config = Path("outputs/garfield_plushie/dig/2024-02-27_170339/config.yml")##v2 128 dim, 7stride
_,pipeline,_,_ = eval_setup(config)
dino_loader = pipeline.datamanager.dino_dataloader


In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor
import moviepy.editor as mpy
# image_path = Path("/home/justin/nerfstudio/data/garfield_plushie/images/frame_00100.png")
video_path = Path("/home/justin/Downloads/garfield_move.mp4")
clip = mpy.VideoFileClip(str(video_path))
#print length of video in seconds
frame = clip.get_frame(7.2)
frame = frame[300:1000,600:1300,:]
pil_image = ToTensor()(Image.fromarray(frame))

# image_path = Path("/home/justin/nerfstudio/data/colorful_mugs_colmap/images/frame_00010.jpg")
# image_path = Path("/home/justin/nerfstudio/data/louvre_statue/images/frame_00020.png")
# pil_image = ToTensor()(Image.open(image_path))
img_pca_feats = dino_loader.get_pca_feats(pil_image.unsqueeze(0)).cuda().squeeze()
cam,data = pipeline.datamanager.next_train(0)
outputs = pipeline.model.get_outputs_for_camera(cam)
which_to_rgb_pca = torch.cat([outputs['dino'].view(-1,img_pca_feats.shape[-1]),img_pca_feats.view(-1,img_pca_feats.shape[-1])],dim=0)
_,_,rgb_pca = torch.pca_lowrank(which_to_rgb_pca.view(-1,which_to_rgb_pca.shape[-1]), q=3, niter=20)
from nerfstudio.utils.colormaps import apply_pca_colormap
_,axs = plt.subplots(1,4,figsize=(20,10))
axs[0].imshow(pil_image.permute(1,2,0).cpu().numpy())
axs[1].imshow(data["image"].cpu().numpy())
axs[2].imshow(apply_pca_colormap(img_pca_feats,rgb_pca).cpu().numpy())
axs[3].imshow(apply_pca_colormap(outputs["dino"],rgb_pca).cpu().numpy())
plt.show()

In [ ]:
pix_coords = [(20,65),(100,65),(80,60),(70,40)]
pix_matches = []
for pc in pix_coords:
    img_pca_feat = img_pca_feats[pc[0],pc[1],:]
    distance_img = (outputs['dino'] - img_pca_feat).pow(2).sum(dim=-1).sqrt()[...,None]
    argmin = torch.argmin(distance_img).cpu().numpy()
    argmin_coords = (argmin//distance_img.shape[1],argmin%distance_img.shape[1])
    pix_matches.append(argmin_coords)
_,axs = plt.subplots(1,2,figsize=(20,10))
#visualize the input click on the original image
axs[0].imshow(apply_pca_colormap(img_pca_feats,rgb_pca).cpu().numpy())
axs[1].imshow(apply_pca_colormap(outputs["dino"],rgb_pca).cpu().numpy())
for i in range(len(pix_matches)):
    color = np.random.rand(3)
    axs[0].scatter(pix_coords[i][1],pix_coords[i][0],c=color,s=300)
    axs[1].scatter(pix_matches[i][1],pix_matches[i][0],c=color,s=300)
plt.show()